In [ ]:
import requests
import gzip
import json

# loading data
response = requests.get("https://sig.infobrisson.fr/france.continental-borders.json.gz")
france = json.loads(gzip.decompress(response.content).decode())

In [ ]:
len(france)

In [ ]:
import proj4py

# from WGS84 to Lambert93
_proj = proj4py.proj4('EPSG:4326', 'EPSG:2154')

def projete(points, reverse=False):
    if isinstance(points[0], (int, float)):
        points = [points]
    func = _proj.inverse if reverse else _proj.forward
    res = [tuple(func(p)) for p in points]
    return res if len(res) != 1 else res[0]

In [ ]:
france_proj = projete(france)

In [ ]:
import numpy as np

def center_of_mass(points):
    points = np.asarray(points)
    assert np.array_equal(points[0, :], points[-1, :])
    px, py = points[:, 0], points[:, 1]
    area = 0.5 * (np.dot(px[:-1], py[1:]) - np.dot(px[1:], py[:-1]))
    return tuple(np.dot(px[:-1] * py[1:] - px[1:] * py[:-1], points[1:, :] + points[:-1, :]) / (6 * area))

In [ ]:
import matplotlib.pyplot as plt

center_proj = center_of_mass(france_proj)
center = projete(center_proj, reverse=True)
print("Coordoonnée du centre de la France :", center)

plt.figure(figsize=(8, 8))
plt.gca(aspect='equal')

plt.plot(*np.asarray(france_proj).T)
plt.plot(*center_proj, 'o')

plt.axis("off")
plt.show()

In [ ]:
import folium

m = folium.Map(location=center, zoom_start=15)

folium.Marker(center, popup='Centre de la France').add_to(m)

m

In [ ]:
m.options['zoom'] = 5

folium.PolyLine(france, color='red', weight=3, opacity=0.7).add_to(m)

m